### Importing Modules

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle

# Scores
from sklearn.model_selection import StratifiedKFold

# Classifiers
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Ignoring Errors
import warnings
warnings.simplefilter('ignore')

In [ ]:
def find_predictions(k, data):
    hierarchical_svm_train, hierarchical_svm_test = [], []
    hierarchical_rf_train, hierarchical_rf_test = [], []
    single_rf_train, single_rf_test = [], []
    
    std = StandardScaler()
    X = np.array(data.iloc[:,2:]) # X is the barcode sequences
    X = std.fit_transform(X)
    species = np.ravel(data.iloc[:,1])
    y = np.ravel(data.iloc[:,0]) # y is the taxonomic classes

    kf = StratifiedKFold(10, random_state=0, shuffle=True)
    
    for train_index, test_index in kf.split(X, species):
        predictions_train_linear_svm, predictions_test_linear_svm = [], []
        predictions_train_single_rf, predictions_test_single_rf = [], []
        predictions_train_hier_rf, predictions_test_hier_rf = [], []
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        species_train, species_test = species[train_index], species[test_index]
        
        
        # Keeping Taxonomy and Species Classes
        Aves_train = X_train[y_train == 'Aves',:]
        Aves_test = X_test[y_test == 'Aves',:]
        
        Chiroptera_train = X_train[y_train == 'Chiroptera',:]
        Chiroptera_test = X_test[y_test == 'Chiroptera',:]
        
        Rodentia_train = X_train[y_train == 'Rodentia',:]
        Rodentia_test = X_test[y_test == 'Rodentia',:]
        
        species_train_aves = species_train[y_train == 'Aves']
        species_train_chiroptera = species_train[y_train == 'Chiroptera']
        species_train_rodentia = species_train[y_train == 'Rodentia']
        
        
        # Classifiers ---------------------------------------------------------------------
        # Linear Svm
        linear_svm_taxonomy = SVC(gamma=1/X_train.shape[0],
                    kernel="linear", probability=True, random_state=0)
        linear_svm_taxonomy.fit(X_train, y_train)
        
        linear_svm_aves = SVC(gamma=1/Aves_train.shape[0],
                kernel="linear", probability=True, random_state=0)
        linear_svm_aves.fit(Aves_train, species_train_aves)
        
        linear_svm_chiroptera = SVC(gamma=1/Chiroptera_train.shape[0],
                kernel="linear", probability=True, random_state=0)
        linear_svm_chiroptera.fit(Chiroptera_train, species_train_chiroptera)
            
        linear_svm_rodentia = SVC(gamma=1/Rodentia_train.shape[0],
                kernel="linear", probability=True, random_state=0)
        linear_svm_rodentia.fit(Rodentia_train, species_train_rodentia)
        
        # Random Forests
        
        single_rf = RandomForestClassifier(n_estimators=10, n_jobs=-1)
        single_rf.fit(X_train, species_train)
        
        
        hierarchical_rf_taxonomy = RandomForestClassifier(n_estimators=10, n_jobs=-1)
        hierarchical_rf_taxonomy.fit(X_train, y_train)
        
        hierarchical_rf_aves = RandomForestClassifier(n_estimators=10, n_jobs=-1)
        hierarchical_rf_aves.fit(Aves_train, species_train_aves)
        
        hierarchical_rf_chiroptera = RandomForestClassifier(n_estimators=10, n_jobs=-1)
        hierarchical_rf_chiroptera.fit(Chiroptera_train, species_train_chiroptera)
        
        hierarchical_rf_rodentia = RandomForestClassifier(n_estimators=10, n_jobs=-1)
        hierarchical_rf_rodentia.fit(Rodentia_train, species_train_rodentia)
    
        
        y_pred_train_linear = linear_svm_taxonomy.predict(X_train)
        y_pred_test_linear = linear_svm_taxonomy.predict(X_test)
        
        predictions_train_single_rf = single_rf.predict(X_train)
        predictions_test_single_rf = single_rf.predict(X_test)
        
        y_pred_train_hier_rf = hierarchical_rf_taxonomy.predict(X_train)
        y_pred_test_hier_rf = hierarchical_rf_taxonomy.predict(X_test)
        
        
        # Predict the Species --------------------------------------------------------------
        for idx in range(X_train.shape[0]):
            # Hierarchical Svm
            if y_pred_train_linear[idx] == 'Aves':
                y_pred_train_species = linear_svm_aves.predict(X_train[idx, :].reshape(1,-1))
                predictions_train_linear_svm.append(y_pred_train_species)
                
            elif y_pred_train_linear[idx] == 'Chiroptera':
                
                y_pred_train_species = linear_svm_chiroptera.predict(X_train[idx, :].reshape(1,-1))
                predictions_train_linear_svm.append(y_pred_train_species)
                
            elif y_pred_train_linear[idx] == 'Rodentia':
                
                y_pred_train_species = linear_svm_rodentia.predict(X_train[idx, :].reshape(1,-1))
                predictions_train_linear_svm.append(y_pred_train_species)
            
            # Hiearachical Rf
            if y_pred_train_hier_rf[idx] == 'Aves':
                y_pred_train_species = hierarchical_rf_aves.predict(X_train[idx, :].reshape(1,-1))
                predictions_train_hier_rf.append(y_pred_train_species)
                
            elif y_pred_train_hier_rf[idx] == 'Chiroptera':
                
                y_pred_train_species = hierarchical_rf_chiroptera.predict(X_train[idx, :].reshape(1,-1))
                predictions_train_hier_rf.append(y_pred_train_species)
                
            elif y_pred_train_hier_rf[idx] == 'Rodentia':
                
                y_pred_train_species = hierarchical_rf_rodentia.predict(X_train[idx, :].reshape(1,-1))
                predictions_train_hier_rf.append(y_pred_train_species)
            
            if idx < X_test.shape[0]:
                # Hierarchical SVM species predictions
                if y_pred_test_linear[idx] == 'Aves':
                
                    y_pred_test_species = linear_svm_aves.predict(X_test[idx, :].reshape(1,-1))
                    predictions_test_linear_svm.append(y_pred_test_species)
                
                elif y_pred_test_linear[idx] == 'Chiroptera':

                    y_pred_test_species = linear_svm_chiroptera.predict(X_test[idx, :].reshape(1,-1))
                    predictions_test_linear_svm.append(y_pred_test_species)

                elif y_pred_test_linear[idx] == 'Rodentia':

                    y_pred_test_species = linear_svm_rodentia.predict(X_test[idx, :].reshape(1,-1))
                    predictions_test_linear_svm.append(y_pred_test_species)
                
                # Hierarchical Rf species predictions
                    
                if y_pred_test_hier_rf[idx] == 'Aves':
                    
                    y_pred_test_species = hierarchical_rf_aves.predict(X_test[idx, :].reshape(1,-1))
                    predictions_test_hier_rf.append(y_pred_test_species)
                
                elif y_pred_test_hier_rf[idx] == 'Chiroptera':

                    y_pred_test_species = hierarchical_rf_chiroptera.predict(X_test[idx, :].reshape(1,-1))
                    predictions_test_hier_rf.append(y_pred_test_species)

                elif y_pred_test_hier_rf[idx] == 'Rodentia':

                    y_pred_test_species = hierarchical_rf_rodentia.predict(X_test[idx, :].reshape(1,-1))
                    predictions_test_hier_rf.append(y_pred_test_species)
                
        # Appending each fold to corresponging data structure
        hierarchical_svm_train.append((species_train, predictions_train_linear_svm)) # true, predicted
        hierarchical_svm_test.append((species_test, predictions_test_linear_svm))
        
        hierarchical_rf_train.append((species_train, predictions_train_hier_rf)) # true, predicted
        hierarchical_rf_test.append((species_test, predictions_test_hier_rf))
        
        single_rf_train.append((species_train, predictions_train_single_rf)) # true, predicted
        single_rf_test.append((species_test, predictions_test_single_rf))

    
    return {'linearsvm':{'train':hierarchical_svm_train, 'test':hierarchical_svm_test},
            'singlerf':{'train':single_rf_train, 'test':single_rf_test},
            'hierarchicalrf':{'train':hierarchical_rf_train, 'test':hierarchical_rf_test}}

### Starting to Robustness Analysis

In [ ]:
mutation_predictions = {}
for k in range(9,11):
    mutation_file = pd.read_csv("mutated-kmerized-datasets/7kmerized.mutated.{}.csv".format(k/10))
    mutation_file = mutation_file.drop(["mutated nucleotide"], 1).dropna(axis=0)
    predictions = find_predictions(k, mutation_file)
    mutation_predictions[str(k)] = predictions
    print(k)

### Storing The Scores

In [ ]:
with open('mutation_predictions9.db','wb') as file:
    pickle.dump(mutation_predictions, file)